In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]
# RESULTS_DIR = 'results_005_lang_id_classification'
RESULTS_DIR = 'results_005_lang_id_classification_dropout_0.1'
GPU_ID = 0

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
bias_category_specification = [
    {
        "category": "utterance",
        "subcategories": [
            {
                "subcategory": "verbal_nod",
                "tags": set(["utt-verbal-nod"])
            },
            {
                "subcategory": "multilingual",
                "tags": set(["utt-multi-lingual", "utt-multi-lingual-named-endity"])
            }
        ]
    },
    {
        "category": "speaker_count",
        "subcategories": [
            {
                "subcategory": "single",
                "tags": set(["spkr-single"])
            },
            {
                "subcategory": "multiple",
                "tags": set(["spkr-mult", "spkr-multi"])
            }
        ]
    },
    {
        "category": "gender",
        "subcategories": [
            {
                "subcategory": "male",
                "tags": set(["spkr-male"])
            },
            {
                "subcategory": "female",
                "tags": set(["spkr-female"])
            },
        ]
    },
    {
        "category": "language",
        "subcategories": [
            {
                "subcategory": "susu",
                "tags": set(["lng-susu"])
            },
            {
                "subcategory": "maninka",
                "tags": set(["lng-maninka"])
            },
            {
                "subcategory": "pular",
                "tags": set(["lng-pular"])
            }
        ]
    },
    {
        "category": "channel",
        "subcategories": [
            {
                "subcategory": "telephone",
                "tags": set(["ct-telephone"])
            },
            {
                "subcategory": "noise",
                "tags": set(["ct-noise"])
            },
            {
                "subcategory": "music",
                "tags": set(["ct-fg-music", "ct-tr-music", "ct-bg-music"])
            }
        ]
    }
]

flat_bias_category_specification = {}

for c in bias_category_specification:
    for sc in c['subcategories']:
        k = f"{c['category']}_{sc['subcategory']}"
        flat_bias_category_specification[k] = sc['tags']

_ = [print(f"{k}: {v}") for k,v in flat_bias_category_specification.items()]

utterance_verbal_nod: {'utt-verbal-nod'}
utterance_multilingual: {'utt-multi-lingual-named-endity', 'utt-multi-lingual'}
speaker_count_single: {'spkr-single'}
speaker_count_multiple: {'spkr-multi', 'spkr-mult'}
gender_male: {'spkr-male'}
gender_female: {'spkr-female'}
language_susu: {'lng-susu'}
language_maninka: {'lng-maninka'}
language_pular: {'lng-pular'}
channel_telephone: {'ct-telephone'}
channel_noise: {'ct-noise'}
channel_music: {'ct-bg-music', 'ct-fg-music', 'ct-tr-music'}


In [5]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [6]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label, tag_set
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels, audio_tags = zip(*data)

## Inspect label counts

In [7]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [8]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels, audio_tags = zip(*balanced_data)

In [9]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


## Inspect bias category counts in balanded data

In [10]:
def inspect_bias_category_counts():
    for name, tags in flat_bias_category_specification.items():
        count = len([ts for ts in audio_tags if len(tags.intersection(ts))>0])
        print(name, count, "/", len(audio_tags))
            
inspect_bias_category_counts()

utterance_verbal_nod 48 / 84
utterance_multilingual 21 / 84
speaker_count_single 25 / 84
speaker_count_multiple 58 / 84
gender_male 81 / 84
gender_female 15 / 84
language_susu 28 / 84
language_maninka 28 / 84
language_pular 28 / 84
channel_telephone 27 / 84
channel_noise 21 / 84
channel_music 24 / 84


# Prepare 10 cross validation folds

In [11]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [12]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [13]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [14]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [15]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [16]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [17]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [19]:
dropout_p = 0.1
class LangIdCNN_Std1(nn.Module):
    def __init__(self):
        super(LangIdCNN_Std1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.std(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.std(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.std(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

class LangIdCNN_Std2(nn.Module):
    def __init__(self):
        super(LangIdCNN_Std2, self).__init__()
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.std(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.std(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.std(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

class LangIdCNN_Mean1(nn.Module):
    def __init__(self):
        super(LangIdCNN_Mean1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x
    
    
class LangIdCNN_Mean2(nn.Module):
    def __init__(self):
        super(LangIdCNN_Mean2, self).__init__()
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

# Train Classification Models

In [20]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']    

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)
    train_tags = np.take(audio_tags, train_indices, axis=0)
    train_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        train_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in train_tags]

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    test_tags = np.take(audio_tags, test_indices, axis=0)
    test_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        test_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in test_tags]
        
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name, batch_size)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [21]:
def train_on_fold(model_class, fold_id, feature_name, batch_size, epochs):
    device = torch.device(f"cuda:{GPU_ID}")
    torch.manual_seed(0)
    results = {}
    
    model = model_class().to(device)

    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model_class(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_n = len(true_train_classes)
        
        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
        train_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes, sample_weight=sw)
            for category, sw in train_bias_category_labels.items()
        }
        


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_n = len(true_test_classes)
        
        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
        test_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes, sample_weight=sw)
            for category, sw in test_bias_category_labels.items()
        }

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'epoch': epoch,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_n': train_n,
            'test_n': test_n,
            
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))
        
    del model
    return results
    

In [22]:
import csv
from pathlib import Path

def save_results(model_name, all_folds_results):
    for result_entry in all_folds_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):

                writer.writerow(result_entry['epochs'][epoch])

In [ ]:
model_classes = [
    LangIdCNN_Std1,
    LangIdCNN_Mean1,
    LangIdCNN_Std2,
    LangIdCNN_Mean2
]

for model_class in model_classes:
    all_folds_results = []
    for fold_index in cv_folds:
                    
        for feature_name in raw_features:
            print(f"{model_class.__name__} using {feature_name} on fold#{fold_index}")
            resutls = train_on_fold(model_class, fold_index, feature_name, batch_size=100, epochs=1000)
            all_folds_results.append({
                'fold_index': fold_index,
                'feature_name': feature_name,
                'epochs': resutls
            })
            save_results(model_class.__name__, all_folds_results)

LangIdCNN_Std1 using wav2vec_features-c on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 1, 2996]           1,537           1,537
         Dropout-2       [10, 1, 2996]               0               0
       AvgPool1d-3       [10, 1, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              12              12
         Dropout-5       [10, 3, 1496]               0               0
       AvgPool1d-6        [10, 3, 748]               0               0
          Conv1d-7        [10, 3, 746]              30              30
         Dropout-8        [10, 3, 746]               0               0
       AvgPool1d-9        [10, 3, 373]               0               0
         Conv1d-10        [10, 3, 371]              30              30
        Dropout-11        [10, 3, 371]               0               0
      AvgPool1d-12        

Epoch: 830. Train Loss: 0.07098. Test Loss: 2.378. Train Acc: 1.0. Test Acc:0.4848
Epoch: 840. Train Loss: 0.06822. Test Loss: 2.392. Train Acc: 1.0. Test Acc:0.4848
Epoch: 850. Train Loss: 0.06448. Test Loss: 2.44. Train Acc: 1.0. Test Acc:0.4848
Epoch: 860. Train Loss: 0.06213. Test Loss: 2.45. Train Acc: 1.0. Test Acc:0.4848
Epoch: 870. Train Loss: 0.07307. Test Loss: 2.471. Train Acc: 1.0. Test Acc:0.4848
Epoch: 880. Train Loss: 0.06581. Test Loss: 2.477. Train Acc: 1.0. Test Acc:0.4848
Epoch: 890. Train Loss: 0.051. Test Loss: 2.518. Train Acc: 1.0. Test Acc:0.4848
Epoch: 900. Train Loss: 0.05267. Test Loss: 2.531. Train Acc: 1.0. Test Acc:0.4545
Epoch: 910. Train Loss: 0.05045. Test Loss: 2.554. Train Acc: 1.0. Test Acc:0.4848
Epoch: 920. Train Loss: 0.0495. Test Loss: 2.545. Train Acc: 1.0. Test Acc:0.4848
Epoch: 930. Train Loss: 0.05351. Test Loss: 2.567. Train Acc: 1.0. Test Acc:0.4848
Epoch: 940. Train Loss: 0.04383. Test Loss: 2.593. Train Acc: 1.0. Test Acc:0.4848
Epoch: 95

Epoch: 640. Train Loss: 0.1325. Test Loss: 1.049. Train Acc: 1.0. Test Acc:0.6667
Epoch: 650. Train Loss: 0.1293. Test Loss: 1.095. Train Acc: 1.0. Test Acc:0.6667
Epoch: 660. Train Loss: 0.1105. Test Loss: 1.056. Train Acc: 1.0. Test Acc:0.6667
Epoch: 670. Train Loss: 0.1158. Test Loss: 1.071. Train Acc: 1.0. Test Acc:0.6667
Epoch: 680. Train Loss: 0.09436. Test Loss: 1.06. Train Acc: 1.0. Test Acc:0.6667
Epoch: 690. Train Loss: 0.09023. Test Loss: 1.065. Train Acc: 1.0. Test Acc:0.6667
Epoch: 700. Train Loss: 0.09762. Test Loss: 1.066. Train Acc: 1.0. Test Acc:0.6667
Epoch: 710. Train Loss: 0.09291. Test Loss: 1.054. Train Acc: 1.0. Test Acc:0.6364
Epoch: 720. Train Loss: 0.09395. Test Loss: 1.063. Train Acc: 1.0. Test Acc:0.6667
Epoch: 730. Train Loss: 0.08227. Test Loss: 1.074. Train Acc: 1.0. Test Acc:0.6667
Epoch: 740. Train Loss: 0.07584. Test Loss: 1.076. Train Acc: 1.0. Test Acc:0.6667
Epoch: 750. Train Loss: 0.06853. Test Loss: 1.085. Train Acc: 1.0. Test Acc:0.6667
Epoch: 76

Epoch: 460. Train Loss: 0.1117. Test Loss: 0.9381. Train Acc: 1.0. Test Acc:0.6667
Epoch: 470. Train Loss: 0.09912. Test Loss: 0.9612. Train Acc: 1.0. Test Acc:0.6667
Epoch: 480. Train Loss: 0.09534. Test Loss: 0.9779. Train Acc: 1.0. Test Acc:0.6667
Epoch: 490. Train Loss: 0.09227. Test Loss: 0.9919. Train Acc: 0.9804. Test Acc:0.6667
Epoch: 500. Train Loss: 0.07382. Test Loss: 1.001. Train Acc: 1.0. Test Acc:0.6364
Epoch: 510. Train Loss: 0.07019. Test Loss: 1.021. Train Acc: 1.0. Test Acc:0.6364
Epoch: 520. Train Loss: 0.06245. Test Loss: 1.028. Train Acc: 1.0. Test Acc:0.6667
Epoch: 530. Train Loss: 0.05525. Test Loss: 1.045. Train Acc: 1.0. Test Acc:0.6667
Epoch: 540. Train Loss: 0.05012. Test Loss: 1.056. Train Acc: 1.0. Test Acc:0.6364
Epoch: 550. Train Loss: 0.04978. Test Loss: 1.067. Train Acc: 1.0. Test Acc:0.6364
Epoch: 560. Train Loss: 0.0436. Test Loss: 1.076. Train Acc: 1.0. Test Acc:0.6364
Epoch: 570. Train Loss: 0.04014. Test Loss: 1.089. Train Acc: 1.0. Test Acc:0.6667

Epoch: 280. Train Loss: 0.6569. Test Loss: 0.9627. Train Acc: 0.6667. Test Acc:0.5152
Epoch: 290. Train Loss: 0.6303. Test Loss: 0.9465. Train Acc: 0.6667. Test Acc:0.5152
Epoch: 300. Train Loss: 0.6137. Test Loss: 0.9479. Train Acc: 0.6667. Test Acc:0.5455
Epoch: 310. Train Loss: 0.5841. Test Loss: 0.9529. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 320. Train Loss: 0.5651. Test Loss: 0.9525. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 330. Train Loss: 0.5481. Test Loss: 0.9556. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 340. Train Loss: 0.5472. Test Loss: 0.9628. Train Acc: 0.6863. Test Acc:0.5455
Epoch: 350. Train Loss: 0.5286. Test Loss: 0.9662. Train Acc: 0.7451. Test Acc:0.5455
Epoch: 360. Train Loss: 0.5148. Test Loss: 0.9625. Train Acc: 0.7059. Test Acc:0.5455
Epoch: 370. Train Loss: 0.4973. Test Loss: 0.9687. Train Acc: 0.7647. Test Acc:0.5455
Epoch: 380. Train Loss: 0.4959. Test Loss: 0.9771. Train Acc: 0.7255. Test Acc:0.5455
Epoch: 390. Train Loss: 0.4863. Test Loss: 0.9822. Tra

...

Epoch: 30. Train Loss: 1.113. Test Loss: 1.085. Train Acc: 0.2353. Test Acc:0.4848
Epoch: 40. Train Loss: 1.102. Test Loss: 1.097. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 50. Train Loss: 1.094. Test Loss: 1.107. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 60. Train Loss: 1.085. Test Loss: 1.114. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 70. Train Loss: 1.076. Test Loss: 1.119. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 80. Train Loss: 1.064. Test Loss: 1.12. Train Acc: 0.3922. Test Acc:0.2424
Epoch: 90. Train Loss: 1.049. Test Loss: 1.117. Train Acc: 0.451. Test Acc:0.2727
Epoch: 100. Train Loss: 1.028. Test Loss: 1.108. Train Acc: 0.4706. Test Acc:0.303
Epoch: 110. Train Loss: 1.001. Test Loss: 1.092. Train Acc: 0.4902. Test Acc:0.2727
Epoch: 120. Train Loss: 0.9682. Test Loss: 1.068. Train Acc: 0.4902. Test Acc:0.3636
Epoch: 130. Train Loss: 0.9291. Test Loss: 1.037. Train Acc: 0.5098. Test Acc:0.4242
Epoch: 140. Train Loss: 0.8873. Test Loss: 1.002. Train Acc: 0.549. Test Acc:0.4242


LangIdCNN_Mean2 using retrained-wav2vec_features-z on fold#1
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2998]           1,539           1,539
          Conv1d-2       [10, 1, 2996]              10              10
         Dropout-3       [10, 1, 2996]               0               0
       AvgPool1d-4       [10, 1, 1498]               0               0
          Conv1d-5       [10, 3, 1496]              12              12
         Dropout-6       [10, 3, 1496]               0               0
       AvgPool1d-7        [10, 3, 748]               0               0
          Conv1d-8        [10, 3, 746]              30              30
         Dropout-9        [10, 3, 746]               0               0
      AvgPool1d-10        [10, 3, 373]               0               0
         Conv1d-11        [10, 3, 371]              30              30
        Dropout

Epoch: 820. Train Loss: 0.004515. Test Loss: 1.486. Train Acc: 1.0. Test Acc:0.8182
Epoch: 830. Train Loss: 0.004192. Test Loss: 1.495. Train Acc: 1.0. Test Acc:0.8182
Epoch: 840. Train Loss: 0.003825. Test Loss: 1.5. Train Acc: 1.0. Test Acc:0.7879
Epoch: 850. Train Loss: 0.003715. Test Loss: 1.515. Train Acc: 1.0. Test Acc:0.8182
Epoch: 860. Train Loss: 0.003815. Test Loss: 1.524. Train Acc: 1.0. Test Acc:0.7879
Epoch: 870. Train Loss: 0.003579. Test Loss: 1.525. Train Acc: 1.0. Test Acc:0.7879
Epoch: 880. Train Loss: 0.003516. Test Loss: 1.535. Train Acc: 1.0. Test Acc:0.7879
Epoch: 890. Train Loss: 0.003254. Test Loss: 1.552. Train Acc: 1.0. Test Acc:0.8182
Epoch: 900. Train Loss: 0.003211. Test Loss: 1.556. Train Acc: 1.0. Test Acc:0.7879
Epoch: 910. Train Loss: 0.00316. Test Loss: 1.563. Train Acc: 1.0. Test Acc:0.7879
Epoch: 920. Train Loss: 0.002979. Test Loss: 1.578. Train Acc: 1.0. Test Acc:0.7879
Epoch: 930. Train Loss: 0.002767. Test Loss: 1.587. Train Acc: 1.0. Test Acc:0.

Epoch: 630. Train Loss: 0.005298. Test Loss: 6.053. Train Acc: 1.0. Test Acc:0.4848
Epoch: 640. Train Loss: 0.005014. Test Loss: 6.023. Train Acc: 1.0. Test Acc:0.4848
Epoch: 650. Train Loss: 0.004911. Test Loss: 6.146. Train Acc: 1.0. Test Acc:0.4848
Epoch: 660. Train Loss: 0.004445. Test Loss: 6.265. Train Acc: 1.0. Test Acc:0.4848
Epoch: 670. Train Loss: 0.003796. Test Loss: 6.298. Train Acc: 1.0. Test Acc:0.4848
Epoch: 680. Train Loss: 0.00384. Test Loss: 6.319. Train Acc: 1.0. Test Acc:0.4848
Epoch: 690. Train Loss: 0.003608. Test Loss: 6.325. Train Acc: 1.0. Test Acc:0.4848
Epoch: 700. Train Loss: 0.003377. Test Loss: 6.35. Train Acc: 1.0. Test Acc:0.4848
Epoch: 710. Train Loss: 0.003412. Test Loss: 6.463. Train Acc: 1.0. Test Acc:0.4848
Epoch: 720. Train Loss: 0.003069. Test Loss: 6.558. Train Acc: 1.0. Test Acc:0.4848
Epoch: 730. Train Loss: 0.003216. Test Loss: 6.63. Train Acc: 1.0. Test Acc:0.4848
Epoch: 740. Train Loss: 0.002724. Test Loss: 6.651. Train Acc: 1.0. Test Acc:0.

Epoch: 430. Train Loss: 0.2343. Test Loss: 1.542. Train Acc: 0.9804. Test Acc:0.5758
Epoch: 440. Train Loss: 0.2188. Test Loss: 1.592. Train Acc: 0.9804. Test Acc:0.5152
Epoch: 450. Train Loss: 0.1902. Test Loss: 1.62. Train Acc: 1.0. Test Acc:0.5152
Epoch: 460. Train Loss: 0.1755. Test Loss: 1.7. Train Acc: 1.0. Test Acc:0.5152
Epoch: 470. Train Loss: 0.1493. Test Loss: 1.705. Train Acc: 1.0. Test Acc:0.5455
Epoch: 480. Train Loss: 0.1331. Test Loss: 1.821. Train Acc: 1.0. Test Acc:0.5455
Epoch: 490. Train Loss: 0.1146. Test Loss: 1.862. Train Acc: 1.0. Test Acc:0.5455
Epoch: 500. Train Loss: 0.09828. Test Loss: 1.926. Train Acc: 1.0. Test Acc:0.5455
Epoch: 510. Train Loss: 0.08454. Test Loss: 1.994. Train Acc: 1.0. Test Acc:0.5455
Epoch: 520. Train Loss: 0.07154. Test Loss: 2.105. Train Acc: 1.0. Test Acc:0.5455
Epoch: 530. Train Loss: 0.06066. Test Loss: 2.155. Train Acc: 1.0. Test Acc:0.5758
Epoch: 540. Train Loss: 0.04963. Test Loss: 2.224. Train Acc: 1.0. Test Acc:0.5758
Epoch: 5